<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso2/ciclo1/Copia_de_M5U1_Taller_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=1QqjbbEZ1w7xoawV020Jj_R46PKRi6A_e" alt = "Encabezado MLDS" width = "100%">  </img>

# **Taller 1: Clasificación lineal con *Tensorflow***
---

En este taller deberá entrenar modelos de clasificación con regresión logística para el [conjunto de datos de vinos Wine](https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data) del repositorio de la *UCI* usando *Tensorflow*.

Ejecute las siguientes celdas para conectarse a UNCode:

In [1]:
!pip install rlxcrypt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.9/297.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.9 MB/s eta 0:00:00
  Created wheel for imphook: filename=imphook-1.0-py3-none-any.whl size=9421 sha256=a9f648941687cb2bc4d5f395bcc82748ca7b77e8ae7a90d567bc3aafd01a9c78
  Stored in directory: /root/.cache/pip/wheels/dc/e2/a4/fcb3817d09a2eb047b2b08eb58e7d9140041b0f3f415eb1256
Successfully built imphook


In [2]:
!wget --no-cache -O session.pye -q https://raw.githubusercontent.com/JuezUN/INGInious/master/external%20libs/session.pye

In [11]:
import rlxcrypt
import session

grader = session.LoginSequence('DLIAAPCP-GroupMLDS-5-2024-2@c5537983-7643-4080-8bd1-dcb22bcd53be')

Please enter your UNCode username: nelson.ferrucho@gmail.com
Please enter your password: ··········


Ejecute la siguiente celda para importar y configurar las librerías usadas :

In [12]:
# Librerías de utilidad para manipulación y visualización de datos.
!pip install -U scikit-learn
from numbers import Number
import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

from numpy.random import seed
seed(1)

# Ignorar warnings.
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Versiones de las librerías usadas.
!python --version
print('Tensorflow', tf.__version__)

Python 3.10.12
Tensorflow 2.17.0


Esta actividad se realizó con las siguientes versiones:
*  Python 3.9.16
*  Tensorflow 2.12.0

## **Cargar los datos**
---
En el conjunto *Wine* las características de entrada corresponden a diferentes atributos del vino. El conjunto de datos contiene 178 ejemplos sobre los que se tiene la siguiente información :

* Alcohol
* Malic acid
* Ash
* Alcalinity of ash
* Magnesium
* Total phenols
* Flavanoids
* Nonflavanoid phenols
* Proanthocyanins
* Color intensity
* Hue
* OD280/OD315 of diluted wines
* Proline

Hay tres clases de vinos diferentes. `class_0` el cual tiene 59 muestras, `class_1` el cual tiene 71 muestras y `class_2` el cual tiene 48 muestras.

Como en cualquier experimento de _machine learning_, vamos a empezar cargando el conjunto de datos, haciendo particiones de entrenamiento y prueba, y para efectos de esta tarea, nos vamos a quedar solo con dos clases (`class_1` y `class_2`) para hacer clasificación:

In [14]:
!wget --no-cache -O wine.data -q  https://raw.githubusercontent.com/mindlab-unal/mlds5-datasets/main/u1/taller/wine.data?raw=true

In [15]:
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection

# Leer el archivo que contiene los datos:
data =  pd.read_csv('wine.data', sep=",", header=None)
# La etiqueta está consignada en la primera columna:
X_all = np.array(data.iloc[:,1:])
y_all = np.array(data.iloc[:,0])
# Nos quedamos con la clase 2 y 3, y ajustamos las etiquetas para que queden
# como 0 y 1:
X = X_all[np.where((y_all==2)|(y_all==3))]
y = y_all[np.where((y_all==2)|(y_all==3))]-2
# Re-escalamos los datos
scaler = preprocessing.MinMaxScaler((0, 1))
X = scaler.fit_transform(X)
# Y hacemos partición en entrenamiento y prueba
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [34]:
y_train

array([1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0])

Verifiquemos el tamaño de las particiones:

In [16]:
print("Número de muestras de entrenamiento =", X_train.shape[0])
print("Número de muestras de prueba =", X_test.shape[0])
print("Número de características del conjunto de datos =", X_train.shape[1])

Número de muestras de entrenamiento = 83
Número de muestras de prueba = 36
Número de características del conjunto de datos = 13


**Salida esperada**

```
Número de muestras de entrenamiento = 83
Número de muestras de prueba = 36
Número de características del conjunto de datos = 13
```

Tenemos entonces 122 muestras con 13 _features_ para trabajar. Ahora veamos cuántas muestras hay por cada clase:

In [31]:
print("Número de muestras de la clase 2 en entrenamiento =", X_train[np.where(y_train==0)].shape[0])
print("Número de muestras de la clase 3 en entrenamiento =", X_train[np.where(y_train==1)].shape[0])
print("Número de muestras de la clase 2 en prueba =", X_test[np.where(y_test==0)].shape[0])
print("Número de muestras de la clase 3 en prueba =", X_test[np.where(y_test==1)].shape[0])

Número de muestras de la clase 2 en entrenamiento = 50
Número de muestras de la clase 3 en entrenamiento = 33
Número de muestras de la clase 2 en prueba = 21
Número de muestras de la clase 3 en prueba = 15


**Salida esperada**
```
Número de muestras de la clase 2 en entrenamiento = 50
Número de muestras de la clase 3 en entrenamiento = 33
Número de muestras de la clase 2 en prueba = 21
Número de muestras de la clase 3 en prueba = 15
```

Como puede ver, el conjunto de datos está desbalanceado. Vamos entonces a implementar un modelo que compense este desbalance desde la función de pérdida.

## **Modelo de regresión logística con _class_weight_**
---
Una vez se dispone de un conjunto de datos preparado para el entrenamiento, se declara el algoritmo de aprendizaje computacional. En nuestro caso queremos predecir el valor de una variable categórica, es decir, realizar un modelo para **clasificación**.

Sin embargo, el conjunto de datos no está balanceado. Cuando esto sucede, podemos compensar el desbalance dándole más importancia a la clase menos presente. Darle más importancia a una clase se logra asignando un peso por cada clase dentro de la función de pérdida del modelo.

Supongamo que tenemos un problema desbalanceado de clasificación binario con etiquetas $0$ y $1$. Supongamos que $n_0$ es el número de elementos de la clase $0$ y $n_1$ es el número de elementos de la clase $1$. Una elección convencional sobre los pesos que se le deben asignar a cada clase es :

$$w_0=\dfrac{n_0+n_1}{2n_0},$$

$$w_1=\dfrac{n_0+n_1}{2n_1}.$$

Y estos pesos se incorporan a la función de pérdida de la siguiente forma :

$$\mathcal{L}(\vec{w})=-\frac{1}{N}\sum_{i=1}^{N}[w_1 y_i\log(\tilde{y}_i)+w_0(1-y_i)\log(1-\tilde{y}_i)],$$

¿Cómo funciona? Supongamos que hay 100 muestras de la clase $0$ y 50 de la clase $1$. Es decir, $n_0=100$ y $n_1=50$. Entonces $w_0=0.75$ y $w_1=1.5$. Así, el peso de la clase $1$ es el doble del peso de la clase $0$. Cuando el modelo no clasifica bien una muestra de la clase $1$, la penalidad se multiplica por $w_1$. Es decir, el modelo entiende que es más grave equivocarse con los datos de la clase $1$, y de esa manera compensa el que sean menos muestras que las de la clase $0$, tratanto de previnir cualquier tipo de sesgo en el modelo final.



> **La tarea es incremental, por lo tanto es recomendable resolver los puntos en orden**

## **1. Calcular el peso de cada clase**
---

Complete la función **`class_weights`** para que calcule los pesos que el modelo tiene que darle a cada clase según el desbalance de los datos.

**Entrada** :

* **`y`**: un `numpy.ndarray` de tamaño `(m,)`; es decir, el vector de etiquetas de los datos de entrenamiento. $m$ el número de muestras del conjunto de entrenamiento.

**Salida** :

* **`weights_list`** : `list`, una lista con los pesos (tipo `float`) de cada clase, en orden.

In [44]:
# FUNCIÓN CALIFICADA class_weights:

def class_weights(y):
    # Reemplazar con respuesta
    # Calcular el número de elementos en cada clase
    n0 = np.sum(y == 0)
    n1 = np.sum(y == 1)

    # Calcular los pesos
    w_0 = (n0 + n1) / (2 * n0)
    w_1 = (n0 + n1) / (2 * n1)

    weights_list = [w_0, w_1]
    return weights_list

In [45]:
#TEST_CELL

class_weights_list = class_weights(y_train)
print(np.round(class_weights_list, 5))

[0.83    1.25758]


**Salida esperada**:

```python
[0.83    1.25758]
```

### **Evaluar código**

In [46]:
grader.run_test("Test 1_1", globals())

Test 1_1


In [47]:
grader.run_test("Test 1_2", globals())

Test 1_2


## **2. Binary cross-entropy**
---
Usaremos el siguente modelo de regresión logística (ejecute la siguiente celda de código para continuar):

In [40]:
# Definimos el modelo de regresión logística
def log_reg(w, b, X):
    return 1/(1+tf.math.exp(-(tf.matmul(X, w) + b)))

Ahora, complete la función **`weighted_bce`** para que retorne el valor de la entropia cruzada, teniendo en cuenta los pesos de cada clase.

**Entrada**:

* **`y_true`** : `tf.Tensor`, un tensor  de tamaño `(m,1)` con las etiquetas reales de los datos.
* **`y_pred`** : `tf.Tensor`, un tensor  de tamaño `(m,1)` con las etiquetas predichas por el modelo.
* **`class_weights`** : `list`, una lista con los pesos asociados a cada clase.

**Salida**:

* **`w_bce`** : `tf.Tensor`, un escalar de tensorflow.

In [52]:
# FUNCIÓN CALIFICADA weighted_bce:
def weighted_bce(y_true, y_pred, class_weights):
    # Reemplazar con respuesta
    # Convertimos y_true a float para realizar las operaciones
    y_true = tf.cast(y_true, dtype=tf.float32)

    # Calculamos la entropía cruzada sin ponderar
    bce = y_true * tf.math.log(y_pred ) + (1 - y_true) * tf.math.log(1 - y_pred)

    # Aplicamos los pesos según la clase (usamos y_true para seleccionar el peso)
    weights = y_true * class_weights[1] + (1 - y_true) * class_weights[0]

    # Multiplicamos las pérdidas por los pesos y sacamos el promedio
    w_bce = -tf.math.reduce_mean(weights * bce)
    return w_bce

Use las siguientes celdas para probar su modelo:

In [53]:
#TEST_CELL

class_weights_example = class_weights(y_train)
w_test=np.array([[1.0],[-1.0],[1.0],[1.0],[-1.0],[1.0],[1.0],[-1.0],[-1.0],[1.0],[1.0],[-1.0],[1.0]])
b_test= 0.5
X_t = tf.constant(X_train, dtype=tf.float32)
Y_t = tf.constant(y_train, dtype=tf.float32)
Y_t = tf.expand_dims(Y_t, axis=-1, name=None)
y_true = Y_t
y_pred = log_reg(w_test, b_test, X_t)
print("weighted_bce =", weighted_bce(y_true,y_pred,class_weights_example).numpy())

weighted_bce = 1.0332682


**Salida esperada**:

```python
weighted_bce = 1.0332682
```

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>


* Utilice la función [**`tf.math.reduce_mean`**](https://www.tensorflow.org/api_docs/python/tf/math/reduce_mean) de la guia de _Tensorflow_ **`tf.math`** tal como se vio en el taller guiado.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>


* El uso de una función de pérdida de entropía cruzada ponderada implica el uso de los pesos de cada clase para hallar su valor.

### **Evaluar código**

In [54]:
grader.run_test("Test 2_1", globals())

Test 2_1


In [55]:
grader.run_test("Test 2_2", globals())

Test 2_2


## **3. Función de pérdida**
---
Complete la función **`loss_fun`** para que calcule el valor de la función de pérdida. Esta es la función que optimizaremos.

**Entrada** :

* **`X_t`**: `tf.Tensor`, tensor de tamaño `(m,n)`, correspondiente a la matriz de datos de entrenamiento.
* **`Y_t`**: `tf.Tensor`, tensor de tamaño `(m,1)`, correspondiente a las etiquetas de los datos entrenamiento.
* **`w`**: `tf.Variable`, tensor de parámetros del modelo, de tamaño `(n,1)`.
* **`b`**: `tf.Variable`, escalar de _bias_ del modelo.
* **`class_weights_list`**: `list`, lista de pesos de cada clase.

**Salida** :

* **`bce`**: `tf.Variable`, escalar con el valor de la función de pérdida.


In [61]:
# FUNCIÓN CALIFICADA create_loss_fun:
def loss_fun(X_t, Y_t, w, b, class_weights_list):
    # Reemplazar con respuesta
    y_pred = log_reg(w, b, X_t)
    bce = weighted_bce(Y_t, y_pred, class_weights_list)
    return bce

Use las siguientes celdas para probar su modelo:

In [58]:
#TEST_CELL
class_weights_list = class_weights(y_train)
w = tf.Variable([[1.0],[-1.0],[1.0],[1.0],[-1.0],[1.0],[1.0],[-1.0],[-1.0],[1.0],[1.0],[-1.0],[1.0]])
b = tf.Variable(0.5)
X_t = tf.constant(X_train, dtype=tf.float32)
Y_t = tf.constant(y_train, dtype=tf.float32)
Y_t = tf.expand_dims(Y_t, axis=-1, name=None)
print("init_loss =",loss_fun(X_t, Y_t, w, b,class_weights_list).numpy())

init_loss = 1.0332682


**Salida esperada**:

```python
init_loss = 1.0332682
```

### **Evaluar código**

In [59]:
grader.run_test("Test 3_1", globals())

Test 3_1


In [60]:
grader.run_test("Test 3_2", globals())

Test 3_2


## **4. Optimizador**
---
Complete la función **`optimizer`** para que devuelva un optimizador válido. Deberá específicar el tipo de optimizador y la tasa de aprendizaje que usaremos en el entrenamiento.

**Entrada** :

* **`type_opt`** : `str`, que puede tomar valores entre: **`SGD`**, **`Adam`**, **`RMSprop`**.
* **`learning_rate`** : `float`, correspondiente a la tasa de aprendizaje.

**Salida** :

* **`opt`** : El optimizador con tasa de aprendizaje definida, un objeto tipo `keras.optimizers`.

In [62]:
# FUNCIÓN CALIFICADA optimizer:
def optimizer(type_opt, learning_rate):

    # Reemplazar con respuesta
    # Definimos el optimizador según el tipo especificado
    if type_opt == 'SGD':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif type_opt == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif type_opt == 'RMSprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError(f"Optimizer '{type_opt}' no es válido. Use 'SGD', 'Adam' o 'RMSprop'.")

    return opt

Use las siguientes celdas para probar su modelo:

In [63]:
#TEST_CELL

type_opt_test = 'Adam'
learning_rate_test = 0.5
optimizer(type_opt_test,learning_rate_test)

**Salida esperada**
> Nota: los últimos números de la salida pueden variar.

```
<keras.optimizers.adam.Adam at 0x7f53eddd39d0>
```

### **Evaluar código**

In [64]:
grader.run_test("Test 4_1", globals())

Test 4_1


In [65]:
grader.run_test("Test 4_2", globals())

Test 4_2


## **5. Entrenamiento del modelo**
---
Una vez definido, podemos entrenar el modelo. Complete la función **`train_model`** para que retorne los pesos **`w`**, **`b`**, del modelo entrenado sobre los arreglos **`X_t`** y **`Y_t`**.

**Entrada** :

* **`epochs`** : `int`, el número de iteraciones durante las cuales se realizará el entrenamiento del modelo.
* **`optimizer`** : `keras.optimizers`, el optimizador definido que se usará para minimizar la función de pérdida **`loss_fun`**
* **`X_t`**: `tf.Tensor`, tensor de tamaño `(m,n)`, correspondiente a la matriz de datos de entrenamiento.
* **`Y_t`**: `tf.Tensor`, tensor de tamaño `(m,1)`, correspondiente a las etiquetas de los datos entrenamiento.
* **`class_weights_list`**: `list`, lista de pesos de cada clase.




**Salida** :

* **`losses`** : `list` Una lista con los valores de **`loss_fun`** en cada iteración.
* **`w`**: `tf.Variable`, tensor de parámetros óptimos del modelo, de tamaño `(n,1)`.
* **`b`**: `tf.Variable`, escalar de _bias_ óptimo del modelo.

> **Nota 1**: los valores iniciales de `w` y `b` ya están definidos dentro de la celda de código que debe completar.

> **Nota 2**: Tal como se hizo en la guía, defina una función `create_model(X_t, Y_t)`, que retorna `w, b, loss_fun`

In [76]:
# FUNCIÓN CALIFICADA train_model:
def train_model(epochs, optimizer, X_t, Y_t, class_weights_list):
    def create_model(X_t, Y_t):
        w = tf.Variable(tf.ones(shape=(13,1)))
        b = tf.Variable(0.5)
        def loss_fun():

            y_pred = log_reg(w, b, X_t)
            bce = weighted_bce(Y_t, y_pred, class_weights_list)
            return bce

        return w, b, loss_fun

    w, b, loss_fun = create_model(X_t, Y_t)
    losses = []
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss = loss_fun()
        gradients = tape.gradient(loss, [w, b])
        optimizer.apply_gradients(zip(gradients, [w, b]))
        losses.append(loss_fun().numpy())


    return losses, w, b

In [77]:
#TEST_CELL
losses, w, b = train_model(epochs=2,optimizer=optimizer('RMSprop',0.5),X_t = X_t, Y_t = Y_t, class_weights_list= class_weights_list)
print('Epoch', 0, 'loss =', losses[0])
print('Epoch', 1, 'loss =', losses[1])
print("w[0] =", w[:1][0,0].numpy())
print("b =",b.numpy())

Epoch 0 loss = 2.0266452
Epoch 1 loss = 1.0402095
w[0] = 0.7889205
b = 0.049995422


**Salida esperada**:

```
Epoch 0 loss = 2.0266452
Epoch 1 loss = 1.0402095
w[0] = 0.78892064
b = 0.049995422
```

### **Evaluar código**

In [78]:
grader.run_test("Test 5_1", globals())

Test 5_1


In [79]:
grader.run_test("Test 5_2", globals())

Test 5_2


## **6. Predicciones del modelo**
---
Finalmente, podemos utilizar los datos reservados de la partición de prueba para calcular predicciones y hacer evaluaciones.

Complete la función **`model_predict`** para que retorne las predicciones del modelo entrenado sobre un conjunto de prueba.

**Entrada** :

* **`log_reg`**: función del modelo de regresión logística previamente definido.
* **`w`**: `tf.Variable`, tensor de parámetros óptimos del modelo, de tamaño `(n,1)`.
* **`b`**: `tf.Variable`, escalar de _bias_ óptimo del modelo.
* **`X_test`** : `tf.Tensor`, tensor de tamaño `(l,n)`, correspondiente a la matriz de datos de prueba.

**Salida** :

* **`y_pred`** : Tensor de tamaño `(l,1)`, con las predicciones de **model** para el conjunto de prueba **`X_test`**.

In [84]:
# FUNCIÓN CALIFICADA model_predict:
def model_predict(log_reg, b, w, X_test):
    # Reemplazar con respuesta
    y_pred = log_reg(w, b, X_test)
    return y_pred

Use la siguiente celda para probar su función:

In [85]:
#TEST_CELL
losses, w, b = train_model(epochs=10,optimizer=optimizer('RMSprop',0.5),X_t = X_t, Y_t = Y_t, class_weights_list= class_weights_list)
X_te = tf.constant(X_test, dtype=tf.float32)
Y_te = tf.constant(y_test, dtype=tf.float32)
Y_te = tf.expand_dims(Y_te, axis=-1, name=None)
y_pred = model_predict(log_reg, b, w, X_te)
print("Primeras dos predicciones:\n", y_pred[:2].numpy())
m = tf.keras.metrics.Accuracy()
m.update_state(Y_te, tf.math.round(y_pred))
print("Accuracy sobre X_test después de 10 epochs:", m.result().numpy())

Primeras dos predicciones:
 [[0.92283654]
 [0.06457147]]
Accuracy sobre X_test después de 10 epochs: 0.9166667


**Salida esperada:**

```
Primeras dos predicciones:
 [[0.92283654]
 [0.06457148]]
Accuracy sobre X_test después de 10 epochs: 0.9166667
```

### **Evaluar código**

In [86]:
grader.run_test("Test 6_1", globals())

Test 6_1


In [87]:
grader.run_test("Test 6_2", globals())

Test 6_2


# **Evaluación**

In [88]:
grader.submit_task(globals())

Test 1_1
Test 1_2
Test 2_1
Test 2_2
Test 3_1
Test 3_2
Test 4_1
Test 4_2
Test 5_1
Test 5_2
Test 6_1
Test 6_2


# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](mailto:mrodrigueztr@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*